In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean
import numpy as np 

In [2]:
sc = SparkSession.builder.appName("dt").getOrCreate()

24/11/05 19:37:43 WARN Utils: Your hostname, prateek-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.202.181 instead (on interface enp0s3)
24/11/05 19:37:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/05 19:37:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/05 19:37:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/05 19:37:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = sc.read.csv("Files/iris.csv",header=True,inferSchema=True)
df.show()

+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
|         5.4|        3.7|         1.5|        0.2| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         4.8|        3.0|         1.4|        0.1| Setosa|
|         4.3|        3.0|         1.1| 

In [4]:
df = df.withColumnRenamed('sepal.length','sepal_length') \
       .withColumnRenamed('sepal.width','sepal_width') \
       .withColumnRenamed('petal.length','petal_length') \
       .withColumnRenamed('petal.width','petal_width')
       
df.show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
|         5.4|        3.7|         1.5|        0.2| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         4.8|        3.0|         1.4|        0.1| Setosa|
|         4.3|        3.0|         1.1| 

In [15]:
mean_sepallen = df.select(mean(df["sepal_length"])).collect()[0][0]
mean_sepalwidth = df.select(mean(df["sepal_width"])).collect()[0][0]
mean_petallen = df.select(mean(df["petal_length"])).collect()[0][0]
mean_petalwidth = df.select(mean(df["petal_width"])).collect()[0][0]

df = df.fillna({'sepal_length' : mean_sepallen,
               'sepal_width' : mean_sepalwidth,
               'petal_length' : mean_petallen,
               'petal_width' : mean_petalwidth})


In [16]:
df.groupBy("variety").count().show()

+----------+-----+
|   variety|count|
+----------+-----+
| Virginica|   50|
|    Setosa|   50|
|Versicolor|   50|
+----------+-----+



In [17]:
df.count()

150

In [18]:
df.printSchema()

root
 |-- sepal_length: double (nullable = false)
 |-- sepal_width: double (nullable = false)
 |-- petal_length: double (nullable = false)
 |-- petal_width: double (nullable = false)
 |-- variety: string (nullable = true)



In [19]:
df.describe().show()

24/11/05 19:43:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     NULL|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     NULL|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



In [20]:
df.columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [21]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [22]:
indexer = StringIndexer(inputCol="variety",outputCol="indexed_variety")
indexmodel = indexer.fit(df)

In [23]:
indexdf = indexmodel.transform(df)

In [24]:
indexdf.show()

+------------+-----------+------------+-----------+-------+---------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|indexed_variety|
+------------+-----------+------------+-----------+-------+---------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|            0.0|
|         4.9|        3.0|         1.4|        0.2| Setosa|            0.0|
|         4.7|        3.2|         1.3|        0.2| Setosa|            0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|            0.0|
|         5.0|        3.6|         1.4|        0.2| Setosa|            0.0|
|         5.4|        3.9|         1.7|        0.4| Setosa|            0.0|
|         4.6|        3.4|         1.4|        0.3| Setosa|            0.0|
|         5.0|        3.4|         1.5|        0.2| Setosa|            0.0|
|         4.4|        2.9|         1.4|        0.2| Setosa|            0.0|
|         4.9|        3.1|         1.5|        0.1| Setosa|            0.0|
|         5.

In [27]:
encoder = OneHotEncoder(inputCol="indexed_variety",outputCol="onehot_variety")
encoded_model = encoder.fit(indexdf)
encoded_df = encoded_model.transform(indexdf)
encoded_df.show()

+------------+-----------+------------+-----------+-------+---------------+--------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|indexed_variety|onehot_variety|
+------------+-----------+------------+-----------+-------+---------------+--------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|
|         4.9|        3.0|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|
|         4.7|        3.2|         1.3|        0.2| Setosa|            0.0| (2,[0],[1.0])|
|         4.6|        3.1|         1.5|        0.2| Setosa|            0.0| (2,[0],[1.0])|
|         5.0|        3.6|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|
|         5.4|        3.9|         1.7|        0.4| Setosa|            0.0| (2,[0],[1.0])|
|         4.6|        3.4|         1.4|        0.3| Setosa|            0.0| (2,[0],[1.0])|
|         5.0|        3.4|         1.5|        0.2| Setosa|            0.0| (2,[0],[1.0])|

In [25]:
from pyspark.ml.feature import VectorAssembler

In [28]:
encoded_df.columns

['sepal_length',
 'sepal_width',
 'petal_length',
 'petal_width',
 'variety',
 'indexed_variety',
 'onehot_variety']

In [29]:
assembler = VectorAssembler(inputCols=['sepal_length',
                                       'sepal_width',
                                       'petal_length',
                                       'petal_width'],outputCol="features")

In [30]:
assembled_df = assembler.transform(encoded_df)
assembled_df.show()

+------------+-----------+------------+-----------+-------+---------------+--------------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|indexed_variety|onehot_variety|         features|
+------------+-----------+------------+-----------+-------+---------------+--------------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|            0.0| (2,[0],[1.0])|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|            0.0| (2,[0],[1.0])|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|            0.0| (2,[0],[1.0])|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|            0.0| (2,[0],[1.0])|[5.4,3.9,1.7,0.4]|
|         4.6|     

In [31]:
assembled_df.select('indexed_variety','features').show()

+---------------+-----------------+
|indexed_variety|         features|
+---------------+-----------------+
|            0.0|[5.1,3.5,1.4,0.2]|
|            0.0|[4.9,3.0,1.4,0.2]|
|            0.0|[4.7,3.2,1.3,0.2]|
|            0.0|[4.6,3.1,1.5,0.2]|
|            0.0|[5.0,3.6,1.4,0.2]|
|            0.0|[5.4,3.9,1.7,0.4]|
|            0.0|[4.6,3.4,1.4,0.3]|
|            0.0|[5.0,3.4,1.5,0.2]|
|            0.0|[4.4,2.9,1.4,0.2]|
|            0.0|[4.9,3.1,1.5,0.1]|
|            0.0|[5.4,3.7,1.5,0.2]|
|            0.0|[4.8,3.4,1.6,0.2]|
|            0.0|[4.8,3.0,1.4,0.1]|
|            0.0|[4.3,3.0,1.1,0.1]|
|            0.0|[5.8,4.0,1.2,0.2]|
|            0.0|[5.7,4.4,1.5,0.4]|
|            0.0|[5.4,3.9,1.3,0.4]|
|            0.0|[5.1,3.5,1.4,0.3]|
|            0.0|[5.7,3.8,1.7,0.3]|
|            0.0|[5.1,3.8,1.5,0.3]|
+---------------+-----------------+
only showing top 20 rows



In [32]:
final_df = assembled_df.select('indexed_variety','features')

In [35]:
train_data, test_data = final_df.randomSplit([0.8,0.2])

In [36]:
train_data.count(), test_data.count()

(113, 37)

In [42]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator


In [43]:
df_classifier = DecisionTreeClassifier(labelCol="indexed_variety",
                                       featuresCol="features")

In [44]:
paramgrid = ParamGridBuilder() \
    .addGrid(df_classifier.maxDepth, [2,5,10]) \
        .addGrid(df_classifier.minInstancesPerNode, [1,2,4]) \
            .addGrid(df_classifier.maxBins, [10,20,30]) \
                .addGrid(df_classifier.minInfoGain, [0.1,0.2,0.3]) \
                    .addGrid(df_classifier.impurity, ['gini','entropy']) \
                        .build()

In [51]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexed_variety",
    metricName="accuracy"
)

In [52]:
crossval = CrossValidator(
    estimator=df_classifier,
    estimatorParamMaps=paramgrid,
    evaluator=evaluator,
    numFolds=3
)

In [53]:
cv = crossval.fit(train_data)

24/11/05 19:55:36 WARN CacheManager: Asked to cache already cached data.
24/11/05 19:55:36 WARN CacheManager: Asked to cache already cached data.


In [54]:
cv.bestModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_633bd44e379e, depth=2, numNodes=5, numClasses=3, numFeatures=4

In [55]:
predictions = cv.bestModel.transform(test_data)
predictions.show()

+---------------+-----------------+--------------+--------------------+----------+
|indexed_variety|         features| rawPrediction|         probability|prediction|
+---------------+-----------------+--------------+--------------------+----------+
|            0.0|[4.6,3.4,1.4,0.3]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.6,3.6,1.0,0.2]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.7,3.2,1.6,0.2]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.8,3.0,1.4,0.1]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.8,3.0,1.4,0.3]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.8,3.4,1.9,0.2]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.9,3.0,1.4,0.2]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.9,3.1,1.5,0.2]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|            0.0|[4.9,3.6,1.4,0.1]|[34.0,0.0,0.0]|       [1.0,0.0,0.0]|       0.0|
|   

In [56]:
accruacy = MulticlassClassificationEvaluator(labelCol="indexed_variety",
                                             metricName="accuracy").evaluate(predictions)

precision = MulticlassClassificationEvaluator(labelCol="indexed_variety",
                                             metricName="weightedPrecision").evaluate(predictions)

In [57]:
accruacy,precision

(0.8918918918918919, 0.903950103950104)

In [60]:
cv.bestModel.featureImportances

SparseVector(4, {3: 1.0})

In [61]:
auc = MulticlassClassificationEvaluator(labelCol="indexed_variety").evaluate(predictions)

In [62]:
auc

0.894036894036894

In [74]:
print(cv.bestModel.getMaxBins)
print(cv.bestModel.maxDepth)
print(cv.bestModel.getImpurity)
print(cv.bestModel.getMinInfoGain)

<bound method _DecisionTreeParams.getMaxBins of DecisionTreeClassificationModel: uid=DecisionTreeClassifier_633bd44e379e, depth=2, numNodes=5, numClasses=3, numFeatures=4>
DecisionTreeClassifier_633bd44e379e__maxDepth
<bound method _TreeClassifierParams.getImpurity of DecisionTreeClassificationModel: uid=DecisionTreeClassifier_633bd44e379e, depth=2, numNodes=5, numClasses=3, numFeatures=4>
<bound method _DecisionTreeParams.getMinInfoGain of DecisionTreeClassificationModel: uid=DecisionTreeClassifier_633bd44e379e, depth=2, numNodes=5, numClasses=3, numFeatures=4>
